# Day 1: Report Repair

## Reduces to the [Knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem).

In [ ]:
# Part 1
# Quadratic Complexity (O(n^2))
# Would get slow fast with larger inputs! Would need to use recursion & memorisation to improve.
import numpy

puzzle_target = 2020
puzzle_input =[2004,1867,1923,1819,1940,1675,1992,1728,2006,1578,1630,1893,1910,1509,1569,1967,1917,1922,1919,1813,1870,370,1617,1600,1729,503,1856,1842,1990,1605,1931,1827,1618,1727,1920,1802,1523,1797,1816,1962,1748,1946,1714,1863,1559,1866,1894,1646,1720,1787,1519,1765,562,1823,1639,1697,544,1938,1681,1477,1778,1718,1853,1632,1651,1694,1683,1911,1692,1997,1745,1873,1750,1795,1924,1724,1596,1726,1979,1869,1740,1847,1951,1541,1755,1991,1680,1612,1903,1691,422,1508,1665,1948,1707,1773,1861,1954,2005,1808,1904,543,1678,2001,1688,1855,1258,1695,1877,1554,1568,1771,1857,1597,1738,577,2010,604,1655,1644,1671,1281,1777,1690,1702,1949,1679,1862,1525,1789,1959,1595,1641,1829,1941,1854,1619,1706,1530,1828,1926,1577,1614,1963,1935,1627,1607,1769,111,1647,1716,1696,1868,1021,1906,1575,1905,1668,1758,1915,1892,1663,2003,1943,1742,1883,1576,1510,1546,1734,814,1367,1902,1698,1912,1818,1615,1851,1564,1719,1952,1616,1988,1768,1957,1744,1858,1705,1794,1944,1973,1960,1887,1804,1913,1770,1825,1737,1799,1532]

puzzle_solution = [(x,y) 
    for (i,x) in enumerate(puzzle_input) 
    for (j,y) in enumerate(puzzle_input) 
    if i<j and x+y == puzzle_target]

if len(puzzle_solution) == 1:
    puzzle_result = numpy.prod([item for sublist in puzzle_solution for item in sublist])
    print(puzzle_result)

802011


In [ ]:
# Part 2
import numpy

puzzle_target = 2020
puzzle_input =[2004,1867,1923,1819,1940,1675,1992,1728,2006,1578,1630,1893,1910,1509,1569,1967,1917,1922,1919,1813,1870,370,1617,1600,1729,503,1856,1842,1990,1605,1931,1827,1618,1727,1920,1802,1523,1797,1816,1962,1748,1946,1714,1863,1559,1866,1894,1646,1720,1787,1519,1765,562,1823,1639,1697,544,1938,1681,1477,1778,1718,1853,1632,1651,1694,1683,1911,1692,1997,1745,1873,1750,1795,1924,1724,1596,1726,1979,1869,1740,1847,1951,1541,1755,1991,1680,1612,1903,1691,422,1508,1665,1948,1707,1773,1861,1954,2005,1808,1904,543,1678,2001,1688,1855,1258,1695,1877,1554,1568,1771,1857,1597,1738,577,2010,604,1655,1644,1671,1281,1777,1690,1702,1949,1679,1862,1525,1789,1959,1595,1641,1829,1941,1854,1619,1706,1530,1828,1926,1577,1614,1963,1935,1627,1607,1769,111,1647,1716,1696,1868,1021,1906,1575,1905,1668,1758,1915,1892,1663,2003,1943,1742,1883,1576,1510,1546,1734,814,1367,1902,1698,1912,1818,1615,1851,1564,1719,1952,1616,1988,1768,1957,1744,1858,1705,1794,1944,1973,1960,1887,1804,1913,1770,1825,1737,1799,1532]

puzzle_solution = [(x,y,z) 
    for (i,x) in enumerate(puzzle_input) 
    for (j,y) in enumerate(puzzle_input)
    for (k,z) in enumerate(puzzle_input) 
    if i<j<k and x+y+z == puzzle_target]

if len(puzzle_solution) == 1:
    puzzle_result = numpy.prod([item for sublist in puzzle_solution for item in sublist])
    print(puzzle_result)

248607374


# Day 2

In [ ]:
# Part 1
from collections import Counter

with open('/content/drive/MyDrive/advent_of_code/day2.txt', 'r') as file:
    puzzle_input = file.read()

puzzle_processing = [[y.split(), z] for y,z in [x.split(':') for x in puzzle_input.split('\n')]]
valid_count = 0

for line in puzzle_processing:
    lower, upper = [int(i.replace("'", '')) for i in line[0][0].split('-')]
    required_char = line[0][1]
    
    password = line[1]
    count_chars = Counter(password)

    for char in count_chars.keys():
        if char == required_char and lower <= count_chars[char] <= upper:
            valid_count += 1

print(valid_count)

477


In [ ]:
# Part 2
with open('/content/drive/MyDrive/advent_of_code/day3.txt', 'r') as file:
    puzzle_input = file.read()

puzzle_processing = [[y.split(), z] for y,z in [x.split(':') for x in puzzle_input.split('\n')]]
valid_count = 0

for line in puzzle_processing:
    pos1, pos2 = [int(i.replace("'", '')) for i in line[0][0].split('-')]
    required_char = line[0][1]
    
    password = line[1].replace(' ', '')

    if password[pos1-1] == required_char or password[pos2-1] == required_char:
        if password[pos1-1] == required_char and password[pos2-1] == required_char:
            continue
        else:
            valid_count += 1

print(valid_count)

686


# Day 3 (review)

In [ ]:
# example = """
# ..##.......
# #...#...#..
# .#....#..#.
# ..#.#...#.#
# .#...##..#.
# ..#.##.....
# .#.#.#....#
# .#........#
# #.##...#...
# #...##....#
# .#..#...#.#"""

# example_list = [list(x) for x in example.split()]

with open('/content/drive/MyDrive/advent_of_code/day3.txt', mode='r') as file:
   puzzle_input = [list(x.strip()) for x in file.readlines()]


def slope_ride(right, down, tree_map):
    # Get the height and width based on the list lengths
    height = len(tree_map)
    width = len(tree_map[0])

    # Set initial values
    trees = 0
    step = 0

    # Walk down the "hill" using range.
    for y in range(down, height, down):
        # We need to track steps down so we know how far to the right.
        step += 1

        # Modulo trick to cycle through the width values and loop back around to the front
        x_axis = (step * right) % width

        # Check if a tree and increment
        if tree_map[y][x_axis] == "#":
            trees += 1

    return trees


def slope_multiplier(slopes, tree_map):
    total = 1

    # Loop through the slopes and calc the trees hit.  Multiply them together.
    for slope in slopes:
        right = slope[0]
        down = slope[1]
        total *= slope_ride(right, down, tree_map)

    return total


print(slope_ride(3, 1, puzzle_input))

slope_list = [
    (1, 1),
    (3, 1),
    (5, 1),
    (7, 1),
    (1, 2)
]

print(slope_multiplier(slope_list, puzzle_input))

191
1478615040


# Day 4

In [ ]:
# PART 1
# example_input = '''ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
# byr:1937 iyr:2017 cid:147 hgt:183cm

# iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
# hcl:#cfa07d byr:1929

# hcl:#ae17e1 iyr:2013
# eyr:2024
# ecl:brn pid:760753108 byr:1931
# hgt:179cm

# hcl:#cfa07d eyr:2025 pid:166559648
# iyr:2011 ecl:brn hgt:59in'''

 
with open('/content/drive/MyDrive/advent_of_code/day4.txt', 'r') as file:
     puzzle_input = file.read()


puzzle_processing = [x.replace('\n', ' ').split() for x in puzzle_input.split('\n\n')]

x = []
for passport in puzzle_processing:
    x.append(dict(details.split(':') for details in passport))

import pandas as pd

df = pd.DataFrame(x)
df['cid'] = df['cid'].fillna(0)
df['count'] = df.count(axis='columns')


print(len(df[(df['count']>7)]))
df.head()

In [ ]:
# PART 2

# byr (Birth Year) - four digits; at least 1920 and at most 2002.
# iyr (Issue Year) - four digits; at least 2010 and at most 2020.
# eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
# hgt (Height) - a number followed by either cm or in:
# If cm, the number must be at least 150 and at most 193.
# If in, the number must be at least 59 and at most 76.
# hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
# ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
# pid (Passport ID) - a nine-digit number, including leading zeroes.
# cid (Country ID) - ignored, missing or not.

df['byr'] = pd.to_numeric(df['byr'])
df['iyr'] = pd.to_numeric(df['iyr'])
df['eyr'] = pd.to_numeric(df['eyr'])

df_processing = df[
         (df['byr']>=1920) & (df['byr']<=2002) & 
         (df['iyr']>=2010) & (df['iyr']<=2020) &
         (df['eyr']>=2020) & (df['eyr']<=2030) &
         (df['hgt'].str.contains(r'[\d]{1,3}(in|cm)', regex=True, na=False)) &
         (df['hcl'].str.contains(r'#[(0-9)|(a-f)]{6}', regex=True, na=False)) &
         (df['ecl'].str.contains(r'(amb)|(blu)|(brn)|(gry)|(grn)|(hzl)|(oth)', regex=True, na=False)) &
         (df['pid'].str.len()<10) & (df['pid'].str.contains(r'[0-9]{9}', regex=True, na=False))
         ].copy()

## df_processing['hgt'] = df['hgt'].str.replace('cm','').str.replace('in','')
## df_processing['hgt'] = pd.to_numeric(df_processing['hgt'])

df_processing['hgt_cm'] = df_processing['hgt'][(df_processing['hgt'].str.contains(r'[\d]{1,3}(cm)', regex=True, na=False))]
df_processing['hgt_in'] = df_processing['hgt'][(df_processing['hgt'].str.contains(r'[\d]{1,3}(in)', regex=True, na=False))]

df_processing['hgt_cm'] = df_processing['hgt_cm'].str.replace('cm','')
df_processing['hgt_in'] = df_processing['hgt_in'].str.replace('in','')

df_processing['hgt_cm'] = pd.to_numeric(df_processing['hgt_cm'])
df_processing['hgt_in'] = pd.to_numeric(df_processing['hgt_in'])

# capture nans
df_processing['hgt_cm'] = df_processing['hgt_cm'].fillna(193)
df_processing['hgt_in'] = df_processing['hgt_in'].fillna(76)

df_final = df_processing[
                         (df_processing['hgt_cm']>=150) & (df_processing['hgt_cm']<=193) &
                         (df_processing['hgt_in']>=59) & (df_processing['hgt_in']<=76)
                         ].copy()

len(df_final)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_processing)

# Day 5

In [ ]:
# EXAMPLE
from math import ceil, floor

puzzle_input = 'FBFBBFFRLR'

def binary_partitioning(binary_choice, search_space):
    partition = [0, (2**len(search_space))-1]
    for ss in search_space:
        if ss == binary_choice:
            partition[1] = floor(partition[0]/2) + floor(partition[1]/2)
        else:
            partition[0] = ceil(partition[0]/2) + ceil(partition[1]/2)
    return min(partition)

row = binary_partitioning('F', puzzle_input[:7])
column = binary_partitioning('L', puzzle_input[7:])

seat_number = (row * 8) + column
print(seat_number)

In [ ]:
# PART 1
with open('/content/drive/MyDrive/advent_of_code/day5.txt', 'r') as file:
     puzzle_input = file.read().splitlines()

seat_numbers = []

for pi in puzzle_input:
    row = binary_partitioning('F', pi[:7])
    column = binary_partitioning('L', pi[7:])
    seat_number = (row * 8) + column
    seat_numbers.append(seat_number)

print(max(seat_numbers))

In [ ]:
# PART 2
seat_numbers.sort()

# Manual check:
temp1 = [i for i in range(min(seat_numbers), max(seat_numbers))]
print(seat_numbers)
print(temp1)

# Using python XOR operator:
# - will remove the duplicates in each list
# - will show differences
print(set(seat_numbers) ^ set(temp1))

# ALT 1:
s = set(seat_numbers)
y = [i for i in temp1 if i not in s]
print(y)

# ALT 2:
print(list(set(temp1) - set(seat_numbers))) # order of lists matters

# ALT 3
temp2 = seat_numbers.copy()
temp3 = [item for item in temp1 if item not in temp2]
print(temp3)

[89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 

# Day 6

In [ ]:
# Example 1
puzzle_input = '''abc

a
b
c

ab
ac

a
a
a
a

b'''

z = ["".join(set(i.replace('\n',''))) for i in puzzle_input.split('\n\n')]
print(len("".join(z)))

11


In [ ]:
# Part 1
with open('/content/drive/MyDrive/advent_of_code/day6.txt', 'r') as file:
    puzzle_input = file.read()

print(len("".join(["".join(set(i.replace('\n',''))) for i in puzzle_input.split('\n\n')])))

6291


In [ ]:
# Example 2
from collections import Counter
puzzle_input = '''abc

a
b
c

ab
ac

a
a
a
a

b'''

puzzle_processing = [i.split('\n') for i in puzzle_input.split('\n\n')]

result = 0

for group in puzzle_processing:
    group_cnt = len(group)
    answers = Counter()

    for ans in ''.join(group):
        answers[ans] += 1
    
    for ans in answers.items():
        if ans[1] % group_cnt == 0:
            x += 1

print(result)

6


In [ ]:
# Part 2
from collections import Counterl

with open('/content/drive/MyDrive/advent_of_code/day6.txt', 'r') as file:
    puzzle_input = file.read()

puzzle_processing = [i.split('\n') for i in puzzle_input.split('\n\n')]
result = 0

for group in puzzle_processing:
    group_cnt = len(group)
    answers = Counter()

    for ans in ''.join(group):
        answers[ans] += 1
    
    for ans in answers.items():
        if ans[1] % group_cnt == 0:
            result += 1

print(result)

3052


# Day 7

In [ ]:
class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append(end)
            else:
                self.graph_dict[start] = [end]
        print("Graph dict:", self.graph_dict)

    def get_paths(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return [path]
        
        if start not in self.graph_dict:
            return []
        
        paths = []

        for node in self.graph_dict[start]:
            if node not in path:
                new_paths = self.get_paths(node, end, path) 
                for p in new_paths:
                    paths.append(p)
        
        return paths

    def get_shortest_path(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return path

        if start not in self.graph_dict:
            return None

        shortest_path = None
        for node in self.graph_dict[start]:
            sp = self.get_shortest_path(node, end, path)
            if sp:
                if shortest_path is None or len(sp) < len(shortest_path):
                    shortest_path = sp

        return shortest_path


    


routes = [
          ('Mumbai', 'Paris'),
          ('Mumbai', 'Dubai'),
          ('Paris', 'Dubai'),
          ('Paris', 'New York'),
          ('Dubai', 'New York'),
          ('New York', 'Toronto')]

route_graph = Graph(routes)

start = 'Paris'
end = 'New York'

print(f'Paths between {start} and {end}: ', route_graph.get_paths(start,end))
print(f'Shortest path between {start} and {end}: ', route_graph.get_shortest_path(start,end))


Graph dict: {'Mumbai': ['Paris', 'Dubai'], 'Paris': ['Dubai', 'New York'], 'Dubai': ['New York'], 'New York': ['Toronto']}
Paths between Paris and New York:  [['Paris', 'Dubai', 'New York'], ['Paris', 'New York']]
Shortest path between Paris and New York:  ['Paris', 'New York']


In [ ]:
# puzzle_input = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
# dark orange bags contain 3 bright white bags, 4 muted yellow bags.
# bright white bags contain 1 shiny gold bag.
# muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
# shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
# dark olive bags contain 3 faded blue bags, 4 dotted black bags.
# vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
# faded blue bags contain no other bags.
# dotted black bags contain no other bags.'''

puzzle_processing = [i for i in puzzle_input.splitlines()]
puzzle_processing = [i.replace('.','').replace('no other bags','').replace(' bags','').replace(' bag','').split(' contain ') for i in puzzle_processing]
puzzle_processing = {key: value.split(', ') for (key, value) in puzzle_processing}

rules = {}

for key, val in puzzle_processing.items():
    if val == ['']:
        continue
    if len(val) > 1:
        val = [i.split(' ', 1) for i in val]
    rules[key] = val

answer = 0
for rule in rules:
    bags = [rule]
    while len(bags) > 0:
        if bags[0] in rules:
            for newbag in rules[bags[0]]:
                if newbag[1] not in bags:
                    bags.append(newbag[1])
        del bags[0]
        if 'shiny gold' in bags:
            answer += 1
            break
print(answer)

# [['light', 'red', 'bags', 'contain', '1', 'bright', 'white', 'bag,', '2', 'muted', 'yellow', 'bags.']]
# [['dark', 'orange', 'bags', 'contain', '3', 'bright', 'white', 'bags,', '4', 'muted', 'yellow', 'bags.']]
# [['bright', 'white', 'bags', 'contain', '1', 'shiny', 'gold', 'bag.']]
# [['muted', 'yellow', 'bags', 'contain', '2', 'shiny', 'gold', 'bags,', '9', 'faded', 'blue', 'bags.']]
# [['shiny', 'gold', 'bags', 'contain', '1', 'dark', 'olive', 'bag,', '2', 'vibrant', 'plum', 'bags.']]
# [['dark', 'olive', 'bags', 'contain', '3', 'faded', 'blue', 'bags,', '4', 'dotted', 'black', 'bags.']]
# [['vibrant', 'plum', 'bags', 'contain', '5', 'faded', 'blue', 'bags,', '6', 'dotted', 'black', 'bags.']]
# [['faded', 'blue', 'bags', 'contain', 'no', 'other', 'bags.']]
# [['dotted', 'black', 'bags', 'contain', 'no', 'other', 'bags.']]

185


In [ ]:
for i in puzzle_processing:
    bag = ' '.join(i[0:2])

    if i[4] == 'no':
        continue
    bag_contents_1 = ' '.join(i[4:7])

    if len(i) != 12:
        graph_dict[bag] = [bag_contents_1.split(' ', 1)]
        continue

    bag_contents_2 = ' '.join(i[8:11])
    graph_dict[bag] = [bag_contents_1.split(' ', 1), bag_contents_2.split(' ', 1)]

In [ ]:
#after parsing the input to a tree, create a recursive function that returns true if the bag or any of its children (recursively) contain the shiny bag.
#Also in part 2, count the bag itself and its children recursively.

#1) the shiny gold bag is directly in its contents, or 2) one of the bags in its contents _contains the shiny gold one_.

# NEED TO LEARN
# graph traversal / tree walking

In [ ]:
class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append(end)
            else:
                self.graph_dict[start] = [end]
        print("Graph dict:", self.graph_dict)

    def get_paths(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return [path]
        
        if start not in self.graph_dict:
            return []
        
        paths = []

        for node in self.graph_dict[start]:
            if node not in path:
                new_paths = self.get_paths(node, end, path) 
                for p in new_paths:
                    paths.append(p)
        
        return paths

    def get_shortest_path(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return path

        if start not in self.graph_dict:
            return None

        shortest_path = None
        for node in self.graph_dict[start]:
            sp = self.get_shortest_path(node, end, path)
            if sp:
                if shortest_path is None or len(sp) < len(shortest_path):
                    shortest_path = sp

        return shortest_path


routes = [
          ('Mumbai', 'Paris'),
          ('Mumbai', 'Dubai'),
          ('Paris', 'Dubai'),
          ('Paris', 'New York'),
          ('Dubai', 'New York'),
          ('New York', 'Toronto')]

route_graph = Graph(routes)

start = 'Paris'
end = 'New York'

print(f'Paths between {start} and {end}: ', route_graph.get_paths(start,end))
print(f'Shortest path between {start} and {end}: ', route_graph.get_shortest_path(start,end))


In [ ]:
with open('/content/drive/MyDrive/advent_of_code/day7.txt', 'r') as file:
    puzzle_input = file.read()


def buildDB():
    db = dict()
    for l in puzzle_input.split("\n"):
        if l != '':
            l = l.replace(',','').split()
            key = "{} {}".format(l[0],l[1])
            t = list()
            l = l[4:]
            if (l != ['no','other','bags.']):
                for i in range(len(l)):
                    if(i % 4 == 0):
                        t.append((int(l[i]),"{} {}".format(l[i+1],l[i+2])))
                if (key not in db):
                    db[key] = t
    return db

db = buildDB()
print(db)
def part1():
    search = ['shiny gold']
    outer,start = set(),0
    while True:
        for i in db.keys(): #
            for j in db[i]:
                if (j[1] in outer or j[1] in search):
                    outer.add(i)
        if (len(outer) > start):
            start = len(outer)
        else:
            break
    print(len(outer))

def part2():
    def getContent(bag):
        c = 1
        if (bag in db):
            for i in db[bag]:
                c += i[0]*getContent(i[1])
        return c
    print(getContent('shiny gold')-1)

part1()
part2()

{'shiny indigo': [(4, 'vibrant lime')], 'clear lime': [(1, 'dotted lime'), (2, 'clear gold')], 'dotted turquoise': [(2, 'shiny green'), (5, 'striped magenta'), (3, 'muted green')], 'bright red': [(1, 'plaid magenta'), (3, 'light cyan')], 'vibrant magenta': [(5, 'drab lime'), (3, 'muted red'), (3, 'wavy turquoise'), (3, 'mirrored salmon')], 'plaid violet': [(4, 'plaid indigo'), (1, 'mirrored beige')], 'muted gray': [(2, 'striped crimson'), (2, 'striped maroon'), (1, 'faded maroon'), (4, 'dim lavender')], 'mirrored crimson': [(5, 'muted lavender')], 'muted gold': [(3, 'striped plum'), (4, 'bright coral')], 'pale black': [(4, 'shiny orange'), (3, 'pale bronze'), (2, 'wavy coral')], 'muted purple': [(5, 'muted brown')], 'clear cyan': [(4, 'muted white')], 'shiny blue': [(5, 'pale magenta'), (3, 'drab coral')], 'posh white': [(3, 'posh brown'), (3, 'striped gold'), (1, 'faded gray')], 'muted beige': [(1, 'pale teal')], 'clear purple': [(4, 'plaid turquoise')], 'pale gold': [(1, 'mirrored li

In [ ]:
import itertools
import copy
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

print(db)

# for i,v in db.items():
#     for j in v:
#         print(i, j)
#df = pd.DataFrame(db)

{'shiny indigo': [(4, 'vibrant lime')], 'clear lime': [(1, 'dotted lime'), (2, 'clear gold')], 'dotted turquoise': [(2, 'shiny green'), (5, 'striped magenta'), (3, 'muted green')], 'bright red': [(1, 'plaid magenta'), (3, 'light cyan')], 'vibrant magenta': [(5, 'drab lime'), (3, 'muted red'), (3, 'wavy turquoise'), (3, 'mirrored salmon')], 'plaid violet': [(4, 'plaid indigo'), (1, 'mirrored beige')], 'muted gray': [(2, 'striped crimson'), (2, 'striped maroon'), (1, 'faded maroon'), (4, 'dim lavender')], 'mirrored crimson': [(5, 'muted lavender')], 'muted gold': [(3, 'striped plum'), (4, 'bright coral')], 'pale black': [(4, 'shiny orange'), (3, 'pale bronze'), (2, 'wavy coral')], 'muted purple': [(5, 'muted brown')], 'clear cyan': [(4, 'muted white')], 'shiny blue': [(5, 'pale magenta'), (3, 'drab coral')], 'posh white': [(3, 'posh brown'), (3, 'striped gold'), (1, 'faded gray')], 'muted beige': [(1, 'pale teal')], 'clear purple': [(4, 'plaid turquoise')], 'pale gold': [(1, 'mirrored li

# Day 8

In [6]:
# # puzzle_input = '''nop +0
# # acc +1
# # jmp +4
# # acc +3
# # jmp -3
# # acc -99
# # acc +1
# # jmp -4
# # acc +6'''


# with open('day8.txt', 'r') as file:
#     puzzle_input = file.read()

# puzzle_processing = [i.split(' ') for i in puzzle_input.split('\n')]
# puzzle_processing = [[i[0],int(i[1])] for i in puzzle_processing]

# swappable_indices = [i for i in range(len(puzzle_processing)-1) if puzzle_processing[i][0] in ('jmp', 'nop')]


# def can_code_complete(code_lines, swap_position=None):
#         accumulator, pointer, line_register = 0, 0, {}
#         while True:
#                 if pointer == len(code_lines):
#                         return True, accumulator
#                 elif pointer in line_register:
#                         return False, accumulator
#                 line_register[pointer] = 1 if pointer not in line_register else line_register[pointer] + 1
#                 instruction = code_lines[pointer][0] if pointer != swap_position else ("nop" if code_lines[pointer][0] == "jmp" else "jmp")
#                 if instruction == "acc":
#                         accumulator += code_lines[pointer][1]
#                 elif instruction == "jmp":
#                         pointer += code_lines[pointer][1]
#                         continue
#                 pointer += 1


# print(can_code_complete(puzzle_processing)[1])
# x = []
# for i in swappable_indices:
#     l = can_code_complete(puzzle_input, swap_position=i)[0]
#     if l is True:
#         x.append(l[1])


# x = [l[1] for index in swappable_indices if (l := can_code_complete(code_lines, swap_position=index))[0] is True][0]

# # code_lines = [((l := line.split(" "))[0], int(l[1])) for line in code.split("\n")]
# # accumulator, pointer, line_register = 0, 0, {}
# swappable_indices = [index for index in range(len(code_lines)-1) if code_lines[index][0] in ("jmp", "nop")]
# print("Part1: accumulator value at first loop start: {}".format(can_code_complete(code_lines)[1]))

In [ ]:
input_list = []

with open('/content/drive/MyDrive/advent_of_code/day8.txt', 'r') as reader:
    for line in reader:
        this_line = str(line.rstrip()).split()
        input_list.append([this_line[0], int(this_line[1])])


def run_program(this_prog, flipped_index):
    accumulator = 0
    current_index = 0
    visited_indices = []

    while True:
        try:
            # Pulling out the operation and argument because nested lists are ugly
            # also we can change the operation without worrying about modifying the list
            operation = this_prog[current_index][0]
            argument = this_prog[current_index][1]
        except IndexError:  # We have successfully terminated the program. Hooray!
            print(f"End of program after flipping index {flipped_index}")
            return accumulator

        if current_index in visited_indices:
            return None  # Nope on outta here -- we got an infinite loop!
        visited_indices.append(current_index)

        # Flip the operation if we're at the target index
        if current_index == flipped_index:
            if operation == 'jmp':
                operation = 'nop'
            elif operation == 'nop':
                operation = 'jmp'

        # Carry out the operations
        if operation == 'acc':
            accumulator += argument
            current_index += 1
        elif operation == 'jmp':
            current_index += argument
        elif operation == 'nop':
            current_index += 1
        else:
            print(f"Unknown argument {argument} found at index {current_index}. Aborting.")
            break

# Gonna loop through each line, and when we find a jmp or an nop, run the program with it "flipped"
i = 0
while True:
    if input_list[i][0] == 'acc':   # No need to "flip" the acc operation, just move right along.
        i += 1
    else:
        result = run_program(input_list, i)
        if result is not None:  # None is returned if its an infinite loop
            print(f"Part 2: {result}")
            break
        else:
            i += 1

End of program after flipping index 381
Part 2: 1976


# Day 9

In [ ]:
# Part 1
from itertools import combinations

with open('/content/drive/MyDrive/advent_of_code/day9.txt', 'r') as file:
     puzzle_input = file.read()

puzzle_processing = [int(i) for i in puzzle_input.splitlines()]

for idx, val in enumerate(puzzle_processing[25:]):
    summed_pairs = [sum(pair) for pair in combinations(puzzle_processing[idx:idx+25], 2)]

    if val not in summed_pairs:
        print(val)

26134589


In [ ]:
# Part 2
class Found(Exception): pass

target = 26134589
step = 2

while True:
    try:
        for idx, val in enumerate(puzzle_processing):
            contiguous_set = puzzle_processing[idx:step]
            if sum(contiguous_set) == target:
                raise Found
        step += 1
    except Found:
        print(max(contiguous_set) + min(contiguous_set))
        break

3535124
404
9042


In [ ]:
# Learning 1
input = list(map(int, open("input/2020/input09.txt", 'r')))
n = len(input)
part1 = input[next(i for i in range(25, n) if all(input[x] + y != input[i] for x in range(i-25, i-1) for y in input[x+1:i]))]
part2 = next(x for x in (next(input[i:j] for j in range(i+1, n) if sum(input[i:j]) >= part1) for i in range(n)) if sum(x) == part1)
print(part1)
print(min(part2) + max(part2))

# js https://davidlozzi.com/2020/12/09/advent-of-code-day-9/
# https://dev.to/qviper/advent-of-code-2020-python-solution-day-9-1knj
# https://dev.to/rpalo/advent-of-code-2020-solution-megathread-day-9-encoding-error-594o

In [ ]:
input = list(map(int, open("/content/drive/MyDrive/advent_of_code/day9.txt", 'r')))
n = len(input)
part1 = input[next(i for i in range(25, n) if all(input[x] + y != input[i] for x in range(i-25, i-1) for y in input[x+1:i]))]
part2 = next(x for x in (next(input[i:j] for j in range(i+1, n) if sum(input[i:j]) >= part1) for i in range(n)) if sum(x) == part1)
print(part1)
print(min(part2) + max(part2))


26134589
3535124


# Day 10 (review)

In [ ]:
pi = '''16
10
15
5
1
11
7
19
6
12
4
'''

pp = list(map(int, pi.splitlines()))
pp.append(0)
pp.append(max(pp) + 3)
pp.sort()
print(pp)

m = max(pp)
print(m)

#low_voltage = filter(lambda x: x <= 3, pp)
low_voltage = [x for x in pp if x <= 3]
print(low_voltage)
new = []

[0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22]
22
[0, 1]


In [ ]:
Assume jolts[] is a vector with the input, plus 0 and max(input)+3
Assume jolts[] is sorted in ascending order
dp[0] = 1
for i = 1 to len(jolts)-1:
    dp[i] = 0
    j = i-1
    while jolts[i] - jolts[j] <= 3:
        dp[i] += dp[j]
        --j
print the last element of dp (i.e., dp[len(dp)-1])


Assume jolts[] is a vector with the input, plus 0 and max(input)+3
Assume jolts[] is sorted in ascending order
dp[0] = 1
sum = 1
j = 0
for i = 1 to len(jolts)-1:
    while jolts[i] - jolts[j] > 3:
        sum -= dp[j]
        ++j
    dp[i] = sum
    sum += dp[i]
print the last element of dp (i.e., dp[len(dp)-1])


In [ ]:

local joltages = {0} -- given input joltages; 0 is added first for the charging outlet
local diff1 = 0
local diff3 = 1 -- starts from 1 as our device's adapter has a built-in joltage of highest +3

-- read the input
for i in io.lines(input) do
    table.insert(joltages, tonumber(i))
end

-- sort the joltages
table.sort(joltages)

-- iterate over all joltages
for i = 1, (#joltages - 1) do
    -- if the difference is 1
    if joltages[i + 1] - joltages[i] == 1 then diff1 = diff1 + 1
    -- else if the difference is 3
    elseif joltages[i + 1] - joltages[i] == 3 then diff3 = diff3 + 1 end
end

print(diff1 * diff3)


In [ ]:
example = '''16
10
15
5
1
11
7
19
6
12
4
'''


joltages = list(map(int, open('/content/drive/MyDrive/advent_of_code/day10.txt', 'r')))
joltages.append(0)

diff1 = 0 # 1 jolt differences cnt
diff3 = 1 # 3 jolt differences cnt
joltages.sort()

for i in range(len(joltages)-1):
    if joltages[i+1] - joltages[i] == 1:
        diff1 = diff1 + 1
    elif joltages[i+1] - joltages[i] == 3:
        diff3 = diff3 + 1

print(diff1 * diff3)

2176


In [ ]:
inp_file = open('/content/drive/MyDrive/advent_of_code/day10.txt')
adapters = []
for line in inp_file:
    adapters.append(int(line))

adapters.append(0)
adapters.sort()
adapters.append(adapters[-1] + 3)


class Graph:
    def __init__(self):
        self.adj_list = []

    def add_edge(self, from_vertex, to_vertex):
        self.adj_list[from_vertex].append(to_vertex)
    
    def add_vertex(self, vertex):
        self.adj_list.append(vertex)

    def __repr__(self):
        return str(self.adj_list)

    def __str__(self):
        return str(self.adj_list)


def part1():
    diffs = [0, 0, 0]

    for i in range(1, len(adapters)):
        diffs[adapters[i] - adapters[i-1] - 1] += 1

    return diffs[0] * diffs[2]



def part2():
    graph = Graph()
    current_vertex = 0
    for i in range(len(adapters)):
        new_edges = []
        for j in range(1,4):
            if i+j < len(adapters) and adapters[i+j] - adapters[i] <= 3:
                new_edges.append(current_vertex+j)

        if len(new_edges) == 1 and (len(graph.adj_list) == 0 or len(graph.adj_list[-1]) == 1):
            continue
        else:
            graph.add_vertex(new_edges)
            current_vertex += 1

    # Get the number of edges at each vertex
    num_of_edges = [len(graph.adj_list[i]) for i in range(len(graph.adj_list))]

    # Start from the second to last edge
    for i in range(len(graph.adj_list)-2, 0, -1):
        for j in range(1, 4):
            # If the current vertex is in a previous vertex
            if i-j >= 0 and i in graph.adj_list[i-j]:
                # Increase the number of edges in the previous vertex
                num_of_edges[i-j] += num_of_edges[i]
                # Remove an edge (because we're essentially doing a substitution)
                num_of_edges[i-j] -= 1
                

    return num_of_edges[0]


print (part1())
print (part2())

2176
18512297918464


# Day 11

In [ ]:
import sys
from typing import Dict, Set, List, Tuple

def parse_seats(lines: List[str]) -> Dict[Tuple[int, int], Set[Tuple[int, int]]]:
    # find all seats in the input, and their connections.
    w, h = len(lines[0]), len(lines)
    seats = {(x, y): {(x, y)} for x in range(w) for y in range(h) if lines[y][x] != '.'}
    for path in generate_paths(w, h):
        previous = None
        for current in path:
            if current in seats:
                if previous:
                    seats[previous].add(current)
                    seats[current].add(previous)
                previous = current
    return seats

def generate_paths(w: int, h: int):
    # to the right
    for y in range(0, h):
        yield ((x, y) for x in range(w))
    # down
    for x in range(0, w):
        yield ((x, y) for y in range(h))
    # down and to the right
    for x0 in range(-h + 1, w):
        yield ((x0 + y, y) for y in range(max(0, -x0), min(h, w - x0)))
    # down and to the left
    for x0 in range(0, w + h):
        yield ((x0 - y, y) for y in range(max(0, x0 - w + 1), min(h, x0 + 1)))

def solve(seats: Dict[Tuple[int, int], Set[Tuple[int, int]]], n: int):
    # assume that all seats are free at the beginning.
    free = {seat for seat, visible in seats.items() if len(visible) > n} # after 2 iterations
    while True:
        # visible = {seat, *neighbours}
        changed = set(seat for seat, visible in seats.items() if visible <= free or seat not in free and len(visible - free) > n)
        if not changed:
            break
        free ^= changed
    return len(seats) - len(free)


lines = [line.rstrip() for line in open('/content/drive/MyDrive/advent_of_code/day11.txt', 'r')]
seats_part2 = parse_seats(lines)
seats_part1 = {(i, j): {(p, q) for (p, q) in visible if max(abs(i-p), abs(j-q)) < 2} for (i, j), visible in seats_part2.items()}
print(f'Part 1: {solve(seats_part1, 4)}')
print(f'Part 2: {solve(seats_part2, 5)}')

Part 1: 2481
Part 2: 2227


In [14]:
from itertools import permutations

def words(letters):
    for n in range(1, len(letters)+1):
        yield from map(''.join, permutations(letters, len(letters)))


word = '12'

for i in words(word):
    print(i)

12
21
12
21
